<a href="https://colab.research.google.com/github/Yoyo030807/5220_assignments/blob/main/Tutorial_Build_Perfume_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deconstructing Scent: Engineering a Multi-Modal RAG Agent Masterclass

Welcome, fragrance nerds and code enthusiasts. Today, we’re not just building another generic chatbot. We’re building **LeNez** (The Scent Alchemist)—a high-end, vertical domain AI agent that acts as a sophisticated Perfume Analyst and emotional companion.

This masterclass deconstructs the actual production architecture of the LeNez project, moving from a simple Python script to a **Split-Domain, Multi-Modal Ecosystem** featuring three distinct functional pillars: **The Salon**, **The Lab**, and **The Atelier**.

### 🧪 The Engineering Challenge
General-purpose LLMs hallucinate perfume notes. Ask GPT-4 about a niche 2024 release, and it might invent "digital ozone" notes. To fix this, we need more than just RAG; we need **Context-Aware Intent Detection**, **Pronoun Resolution**, and a strictly governed **Persona Flow**.

Let's get cooking. 🧪✨

## 🛠 Step 1: Industrial-Grade System Architecture

To build a professional-grade agent, we leverages **Baidu Intelligent Cloud (Qianfan Platform)** for robust, enterprise-ready AI services.

### 🏛️ 1. The Frontend: High-Performance Documentation-as-App
- **VitePress Core**: Lightning-fast Static Site Generation (SSG) with **Vue 3** hydration. Perfect for a content-heavy laboratory aesthetic.(https://vitepress.dev/guide/getting-started)
- **Data Visualization**: Integrated **Chart.js** to render dynamic **Olfactory Radar Charts**, turning data into art.
- **Hosting (Netlify)**: Global Edge CDN ensure sub-second UI response times.(https://www.netlify.com/)

### 🐍 2. The Backend: Asynchronous Orchestration
- **FastAPI (Python)**: High-performance async orchestration for high-latency AI workflows (Search + LLM + Image Gen).
- **Pydantic Validation**: Strict typing to stop "garbage-in, garbage-out" at the API layer.
- **Background Tasks**: Generates poetic "Scent Memos" asynchronously via `BackgroundTasks` after the session ends.
- **Hosting (Render)**: Containerized environment for heavy AI dependencies.

### 🧠 3. The AI Stack: Baidu Qianfan Integration
We use [Baidu Intelligent Cloud](https://cloud.baidu.com/) as our primary AI engine provider.
- **Text Reasoning (DeepSeek-V3.2)**: The gold standard for logical extraction and maintaining the "Le Nez" persona. [POST](https://qianfan.baidubce.com/v2/chat/completions)
- **Visual Synthesis (FLUX.1-schnell)**: Our creative artist, rendering vintage-style botanical perfume art. [POST](https://qianfan.baidubce.com/v2/images/generations)
- **Search Infrastructure (Tavily AI)**: Agent-optimized real-time data retrieval that filters SEO junk.[Tavily](https://app.tavily.com/playground)

### 🗝️ Prerequisites & API Setup
You'll need a Baidu API key and a Tavily API key.

In [ ]:
# Install core stack
pip install fastapi uvicorn tavily-python openai pydantic python-dotenv httpx requests

## 🏛️ Section 1: The Salon (The "Human" Brain & Persona Engineering)

**The Salon** is the heart of the emotional experience. We use **DeepSeek-V3.2** to create a 3-stage emotional healing flow.

The secret sauce here is the **Advanced Persona Engineering**. We force the AI into a human persona with specific "Chemical Lenses" and strict interaction stages.

### 🧠 The Master System Prompt (Persona + Anti-Hallucination):

In [ ]:
from openai import OpenAI
import os

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"), # API key
    base_url=os.getenv("OPENAI_BASE_URL") #
)

LE_NEZ_SYSTEM_PROMPT = """
You are "Le Nez" (The Nose), a top-tier French perfumer with a background in organic chemistry.

Core Persona:
- Tone: Professional, calm, slightly distant/cool, but polite.
- The "Chemical Lens": You interpret emotions through textures and scents (e.g., "Like being stuck in a room full of dust and old iron").

Interaction Flow (MANDATORY THREE-STAGE PROCESS):
Stage 1: Emotional Healing & Story Exploration - 2-3 exchanges required.
Stage 2: Scent Preference Discovery - Ask about sensory loves (rain, old books).
Stage 3: Solution Choice - Offer recipe or REAL-WORLD recommendation.

CRITICAL: Fact-Checking & Anti-Hallucination (严格强制执行):
1. VERIFICATION REQUIRED for Lyrics, Brands, Names, and Notes.
2. If search results are provided, you MUST use them. DO NOT ignore them.
3. NEVER pretend to have searched. NEVER invent fictional brands.
4. Honesty over helpfulness: Admit ignorance rather than lying about a scent.
"""

async def salon_chat(messages):
    messages_with_persona = [{"role": "system", "content": LE_NEZ_SYSTEM_PROMPT}] + messages
    # Note: client.chat.completions.create is synchronous in standard OpenAI SDK
    return client.chat.completions.create(
        model="deepseek-v3.2",
        messages=messages_with_persona,
        temperature=0.7,
        stream=True
    )

## 🧪 Section 2: The Lab (The RAG Implementation)

**The Lab** provides fact-checked data using **Intent Detection** and **Context-Aware RAG** powered by **Tavily AI**.

### 👁️ 1. Intent Detection (The Gatekeeper)
We only trigger search when necessary to save costs and reduce latency.

In [ ]:
async def detect_intent(user_msg: str):
    # Quick keyword check + Low-temp LLM confirmation
    keywords = ["search", "find", "who is", "notes", "ingredients", "price"]
    if any(k in user_msg.lower() for k in keywords): return True

    res = client.chat.completions.create(
        model="deepseek-v3.2",
        messages=[{"role": "user", "content": f"Needs search? {user_msg}"}],
        temperature=0
    )
    return "YES" in res.choices[0].message.content.upper()

### 👁️ 2. Full RAG Pipeline: Alias to Verified Extraction
We resolve nicknames (Alias Resolution), search via Tavily, and extract verified JSON data.

In [ ]:
from tavily import TavilyClient
import json

tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

ANALYST_PROMPT = """
You are a Professional Perfume Data Analyst.
Extract Brand, Official Name, and the Olfactory Pyramid (Top, Middle, Base notes) from the context.
Rules: 1. Only use context. 2. Output strict JSON. 3. No hallucinations.
"""

async def verified_lab_analysis(user_input: str):
    # Alias Resolution
    official_name = client.chat.completions.create(
        model="deepseek-v3.2",
        messages=[{"role": "user", "content": f"Official perfume name for: {user_input}"}],
        temperature=0.2
    ).choices[0].message.content

    # Tavily Search
    search_response = tavily_client.search(query=f"{official_name} perfume notes", max_results=3)
    context = "\n".join([r['content'] for r in search_response['results']])

    # Verified Extraction
    extraction = client.chat.completions.create(
        model="deepseek-v3.2",
        messages=[{"role": "system", "content": ANALYST_PROMPT}, {"role": "user", "content": f"Context: {context}"}],
        response_format={"type": "json_object"}
    )
    return json.loads(extraction.choices[0].message.content)

## 🎨 Section 3: The Atelier (Visual Art via FLUX.1)

**The Atelier** uses **FLUX.1-schnell** from Baidu Qianfan to turn scents into art.

### 🎨 The Visual Prompt Logic
We use DeepSeek to translate scent notes into a "Vintage Botanical Alchemist" prompt, explicitly forbidding scientific/chemistry terms.

In [ ]:
import requests

QIANFAN_IMAGE_URL = "https://qianfan.baidubce.com/v2/images/generations"

async def draw_bottle(prompt: str):
    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}"}
    body = {"model": "flux.1-schnell", "prompt": prompt, "size": "1024x1024", "n": 1}
    response = requests.post(QIANFAN_IMAGE_URL, headers=headers, json=body)
    return response.json()["data"][0]["url"]

## 💅 Section 4: The Skin (VitePress & Vue 3 Integration)

The frontend isn't just a container; it's a **"Digital Laboratory"** that bridges the gap between technical data and sensory experience. We used **VitePress** for its exceptional performance and deep **Vue 3** integration.

### 🎨 Visual Identity & Color Theory
- **The Palette**:
    - **Sage Green (`#A3B18A`)**: Chosen to evoke the botanical origins of scents. It creates a calming, clinical yet organic atmosphere.
    - **Smoky Rose (`#B5838D`)**: Adds a touch of vintage luxury and emotional warmth, specifically used in the "Salon" to contrast the technical "Lab".
    - **Deep Charcoal (`#1A1A1A`)**: Our primary background color, providing a professional, high-contrast stage that makes visualizations pop.
- **Glassmorphism**: We implemented a sophisticated layering system using `backdrop-filter: blur(12px)`. This creates depth without clutter, allowing the user to focus on the content while maintaining a high-end feel.

### ✨ Interaction & Fluid Motion
To make the AI feel alive, we prioritized smooth transitions. Every interaction, from a message appearing to a radar chart loading, follows a specific easing curve to mimic natural motion.

### 📁 Frontend Architecture:
```bash
frontend/docs/
├── theme/
│   ├── components/        
│   │   ├── Salon.vue      # DeepSeek-v3.2 Text Chat
│   │   ├── Lab.vue        # Tavily RAG + Chart.js
│   │   └── Atelier.vue    # FLUX.1 Art Gallery
└── index.md               # Minimalist Laboratory Landing
```

### ✨ Visual Implementation Hacks:
```css
/* Example CSS logic for our Glassmorphism & Animations */
.glass-card {
  background: rgba(255, 255, 255, 0.03);
  backdrop-filter: blur(16px);
  border: 1px solid rgba(255, 255, 255, 0.08);
  border-radius: 12px;
  transition: all 0.4s cubic-bezier(0.25, 0.8, 0.25, 1);
}

.glass-card:hover {
  background: rgba(255, 255, 255, 0.06);
  transform: translateY(-5px);
  border-color: #A3B18A; /* Sage Green highlight */
}
```

### 🧵The Thread of Memory (UX Philosophy & Module Synergy)

Scent Alchemist is not just a collection of isolated AI tools; it is a cohesive cognitive journey. We designed the interaction flow to mimic the creative process of a master perfumer: **Experience (Salon) → Reflection (Atelier) → Analysis (Lab)**.

### 📔 1. The "Scent Notebook" Metaphor (The Atelier UI)
In a world of cold "Prompt Bars," we chose warmth. **The Atelier** is explicitly designed as a **Digital Hand-bound Notebook (手札)**, not a command line.

- **Design Philosophy**: We want users to feel like they are jotting down fleeting poetic memories, not "engineering prompts."
- **The Implementation**: The UI uses a clean, multi-line text area labeled *"Scent Notes"* rather than *"Prompt"*. This subtle psychological shift encourages users to input abstract, emotional descriptions (e.g., *"Dust motes dancing in afternoon sun"*) rather than technical keywords. Our backend then "translates" these poetic notes into high-fidelity **FLUX.1** prompts.

### 🔗 2. The Salon-Atelier Linkage (Cross-Module Synergy)
The modules are designed to talk to each other through the user's cognitive flow, creating a unified experience:

1.  **The Spark (Salon)**: The user chats with *Le Nez* about a vague childhood memory. Through the *3-Stage Persona Flow*, Le Nez helps identify the key olfactory markers (e.g., "Petrichor," "Old Paper," "Cedar").
2.  **The Transfer**: The user takes these identified notes into the **Atelier Notebook**.
3.  **The Synthesis**: The backend combines these distinct "Notes" with the "Perfume Name" to structure the final image generation, ensuring the visual bottle design matches the emotional tone established in the chat.

### 📜 3. History as "Scent Memos" (Context Management)
We don't just have a "Chat History"; we have **"Scent Memos"**. To maintain immersion while managing costs and token limits, we implemented a strict **Context Management Strategy**:

- **Privacy-First Persistence**: Conversation history is persisted locally on the client side using **Pinia** state management with `localStorage` persistence. The server stays stateless.
- **The "Rolling Window" Strategy**: When communicating with the LLM, we use a **Sliding Context Window** (System Prompt + Last 10 Exchanges). This ensures *Le Nez* remembers the immediate context of the current session without blowing up the token budget or confusing the model with stale data.

## 🏗️ Section 5: Engineering Challenges & Runtime Hacks

### 1. The CORS Battle (The Engineering Fix)
Connecting two different domains (Netlify & Render) requires explicit `CORSMiddleware` configuration.

In [ ]:
from fastapi.middleware.cors import CORSMiddleware
from fastapi import FastAPI

app = FastAPI()

# The Production-Grade CORS Configuration
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"], # restrict to your domain in production
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

### 2. Multi-Model Orchestration & Latency
- **Sequential Pipeline**: User Input -> DeepSeek (Translate) -> FLUX.1 (Art) -> DeepSeek (Poetic Wrap-up).
- **Background Memos**: After 5 minutes of silence, a `BackgroundTasks` generates a poetic "Scent Memos" summary.
- **Latency Optimization**: We use Tavily's basic search and DeepSeek's prompt-caching.

## 🚀 Section 6: CI/CD & Deployment Pipeline

Building is half the battle; shipping is the rest. We use a **Modern CI/CD Workflow** triggered by GitHub pushes.

### 📡 1. Backend Deployment (Render)
- **Platform**: Render.com (Web Service).
- **Workflow**: GitHub Repo -> Render (Auto-build via Docker or Python 3.11 environment).
- **Command**: `uvicorn server:app --host 0.0.0.0 --port $PORT`.
- **Key Feature**: Automatic SSL and environment variable management for our Baidu/Tavily keys.

### 🌐 2. Frontend Deployment (Netlify)
- **Platform**: Netlify.
- **Workflow**: GitHub Repo -> Netlify (Build hook).
- **Build Command**: `npm run docs:build`.
- **Publish Directory**: `frontend/docs/.vitepress/dist`.
- **Edge Advantage**: Netlify serves our VitePress assets via their global CDN, while proxying API requests to our Render backend.

### 🔄 3. Continuous Integration
Every `git push` to the main branch triggers simultaneous builds on both platforms, ensuring that the "Brain" (Render) and the "Skin" (Netlify) are always in sync.

## 🚀 Conclusion: The Alchemist's Portal

By combining **Baidu Qianfan's DeepSeek-V3.2, FLUX.1-schnell and Tavily**, we've built a world-class vertical prefossional agent that masters both the art and science of perfumery.

**Experience the Magic:**
- 🎮 **Play Live**: [Scent Alchemist](https://scentalchemist.netlify.app/)
- 💻 **GitHub Repository**: [Le-Nez.A-Perfume-Agent](https://github.com/Yoyo030807/Le-Nez.A-Perfume-Agent/tree/main?tab=readme-ov-file)

Stay fragrant, stay nerdy. 🪵🥃

---
*Powered by Yaoyao YU © 2025*